In [2]:
import json
from typing import List

import pandas as pd
import requests
import re
import string
import sched, time
import csv
from datetime import datetime

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

s = sched.scheduler(time.time, time.sleep)
subreddit = 'all'
limit = 100
timeframe = 'month'  # hour, day, week, month, year, all
listing = 'controversial'  # controversial, best, hot, new, random, rising, top
filename = datetime.now().strftime('new-%Y-%m-%d-%H-%M.csv')
after = ""
sample_str = "Test&[88]%%$$$#$%-+String"


def get_reddit_JSON(subreddit, listing, limit, timeframe):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}'
        request = requests.get(base_url, headers={'User-agent': 'yourbot'})

    except:
        print('An Error Occurred in pulling from Reddit')

    return request.json()

def get_reddit_JSONNext(subreddit, listing, limit, timeframe,after):
    try:
        base_url = f'https://www.reddit.com/r/{subreddit}/{listing}.json?limit={limit}&t={timeframe}&after={after}'
        request = requests.get(base_url, headers={'User-agent': 'yourbot'})

    except:
        print('An Error Occurred in pulling from Reddit')

    return request.json()


def get_results(r):
    myDict = {}
    for post in r['data']['children']:
        myDict[post['data']['title']] = {
            'listingID': removeSpaces(post['data']['title']) + "_" + post['data']['subreddit'],
            'createdDate': post['data']['created_utc'],
            'redditCallType': listing,
            'url': post['data']['permalink'],
            'score': post['data']['score'],
            'ups': post['data']['ups'],
            'downs': post['data']['downs'],
            'title': cleanChars(post['data']['title']),
            'sentimentTitle': sentimentAnalyze(post['data']['title']),
            'sentimentSCore': sentimentScorer(post['data']['title']),
            'sentimentBody': sentimentAnalyze(post['data']['selftext']),
            'comments': post['data']['num_comments'],
            'author': post['data']['author'],
            'subreddit': post['data']['subreddit'],
            'body': cleanChars(post['data']['selftext'])
        }

    df = pd.DataFrame.from_dict(myDict, orient='index')
    #print("Reddit scraped")
    return df

def removeSpaces(sample_str2):
    pattern = r'[^A-Za-z0-9]+'
    titleNoSpaces = re.sub(pattern, '', sample_str2)
    if titleNoSpaces is None:
        titleNoSpaces = "   "
    return titleNoSpaces

def cleanChars(sample_str):

    # Create a regex pattern to match all special characters in string
    pattern = r'[^A-Za-z0-9 ]+'
    # Remove special characters from the string
    newString = re.sub(pattern, ' ', sample_str)
    return newString

def sentimentAnalyze(sample_str3):
    #sentence = "I am very sad today."
    sentence = sample_str3
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    # decide sentiment as positive, negative and neutral
    if sentiment_dict['compound'] >= 0.05:
        sentiment = "Positive"
    elif sentiment_dict['compound'] <= - 0.05:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"
    return sentiment

def sentimentScorer(sample_str3):
    #sentence = "I am very sad today."
    sentence = sample_str3
    sid_obj = SentimentIntensityAnalyzer()
    sentiment_dict = sid_obj.polarity_scores(sentence)
    return sentiment_dict['compound']


r = get_reddit_JSON(subreddit, listing, limit, timeframe)
f = get_results(r)
z = f.copy()
after = r['data']['after']
numCalls = 0
while numCalls < 301:
#while numCalls < 4000:
    k = get_reddit_JSONNext(subreddit, listing, limit, timeframe, after); n = get_results(k); z = pd.concat([z, n]); after = k['data']['after']; numCalls +=1
filename = datetime.now().strftime('con_Month_%Y-%m-%d-%H-%M.csv')
z.to_csv(filename, index=False)

